# Start to try and add pycnocline diagnostics to COAsT

In [1]:
import coast
import numpy as np
import xarray as xr
import dask

#dir = "example_files/"
dir = "/Users/jeff/downloads/"

fn_nemo_dat = 'COAsT_example_NEMO_data.nc'
fn_nemo_dom = 'COAsT_example_NEMO_domain.nc'
fn_altimetry = 'COAsT_example_altimetry_data.nc'

sec = 1
subsec = 96 # Code for '`' (1 below 'a')


In [2]:
import matplotlib.pyplot as plt

In [3]:
#################################################
## ( 1 ) Test Loading and initialising methods ##
#################################################

#-----------------------------------------------------------------------------#
# ( 1a ) Load example NEMO data (Temperature, Salinity, SSH)                  #
#                                                                             #

sci = coast.NEMO() 
sci.load(dir + fn_nemo_dat)

sci_dom = coast.DOMAIN()
sci_dom.load(dir + fn_nemo_dom)

sci.set_command_variables()
sci_dom.set_command_variables()


'Dataset' object has no attribute 'botpres'
'Dataset' object has no attribute 'voctemper'
'Dataset' object has no attribute 'soce'
'Dataset' object has no attribute 'e3t'
'Dataset' object has no attribute 'e3u'
'Dataset' object has no attribute 'e3v'
'Dataset' object has no attribute 'uoce'
'Dataset' object has no attribute 'voce'
'Dataset' object has no attribute 'utau'
'Dataset' object has no attribute 'vtau'


In [4]:
sci.dataset


<xarray.Dataset>
Dimensions:              (axis_nbounds: 2, deptht: 51, time_counter: 7, x: 297, y: 375)
Coordinates:
  * deptht               (deptht) float32 3.038086 9.3671875 ... 5822.174
    nav_lat              (y, x) float32 ...
    nav_lon              (y, x) float32 ...
  * time_counter         (time_counter) datetime64[ns] 2007-01-01T11:58:56 ... 2007-01-31T11:58:56
    time_instant         (time_counter) datetime64[ns] ...
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    deptht_bounds        (deptht, axis_nbounds) float32 ...
    sossheig             (time_counter, y, x) float32 ...
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] ...
    votemper             (time_counter, deptht, y, x) float32 ...
Attributes:
    name:         AMM7_1d_20070101_20070131_25hourm_grid_T
    description:  ocean T grid variables, 25h meaned
    title:        ocean T grid variables, 25h meaned
    Conventions:  CF-1.6
    timeStamp:    2019-Dec-26 04:35:28 GMT
    uuid:         96cae459-d3a1-4f4f-b82b-9259179f95f7
    history:      Tue May 19 12:07:51 2020: ncks -v votemper,sossheig -d time...
    NCO:          4.4.7

In [5]:
# Pick out a North Sea subdomain
ind = sci_dom.subset_indices([50,-5], [70,10])

sci_nwes = sci.isel(y=ind[0], x=ind[1]) #nwes = northwest europe shelf
sci_nwes.set_command_variables()

'Dataset' object has no attribute 'botpres'
'Dataset' object has no attribute 'voctemper'
'Dataset' object has no attribute 'soce'
'Dataset' object has no attribute 'e3t'
'Dataset' object has no attribute 'e3u'
'Dataset' object has no attribute 'e3v'
'Dataset' object has no attribute 'uoce'
'Dataset' object has no attribute 'voce'
'Dataset' object has no attribute 'utau'
'Dataset' object has no attribute 'vtau'


In [6]:
t = sci_nwes.dataset.votemper
t

<xarray.DataArray 'votemper' (time_counter: 7, deptht: 51, y: 226, x: 136)>
[10972752 values with dtype=float32]
Coordinates:
  * deptht        (deptht) float32 3.038086 9.3671875 ... 5617.8516 5822.174
    nav_lat       (y, x) float32 ...
    nav_lon       (y, x) float32 ...
  * time_counter  (time_counter) datetime64[ns] 2007-01-01T11:58:56 ... 2007-01-31T11:58:56
    time_instant  (time_counter) datetime64[ns] 2007-01-02 ... 2007-02-01
Dimensions without coordinates: y, x
Attributes:
    units:               degC
    online_operation:    instant
    interval_operation:  300 s
    interval_write:      1 d
    cell_methods:        time: point (interval: 300 s)

In [7]:
#plt.pcolor( sci_nwes.dataset.votemper[5,30,:,:] )

In [ ]:
def strat(var, grid, ax=ax):
    """Compute the derivative on the provided grid"""

In [7]:
np.shape(sci_nwes.dataset.votemper[5,30,:,:])

(226, 136)

In [7]:
sci_dom.dataset

<xarray.Dataset>
Dimensions:       (t: 1, x: 297, y: 375, z: 51)
Dimensions without coordinates: t, x, y, z
Data variables:
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
    nav_lev       (z) float32 ...
    time_counter  (t) float32 ...
    jpiglo        float32 ...
    jpjglo        float32 ...
    jpkglo        float32 ...
    jperio        float32 ...
    ln_zco        float32 ...
    ln_zps        float32 ...
    ln_sco        float32 ...
    ln_isfcav     float32 ...
    glamt         (t, y, x) float32 -19.888672 -19.777344 ... 12.888672 13.0
    glamu         (t, y, x) float32 ...
    glamv         (t, y, x) float32 ...
    glamf         (t, y, x) float32 ...
    gphit         (t, y, x) float32 40.066406 40.066406 ... 65.00098 65.00098
    gphiu         (t, y, x) float32 ...
    gphiv         (t, y, x) float32 ...
    gphif         (t, y, x) float32 ...
    e1t           (t, y, x) float32 ...
    e1u           (t, y, x) float32 ...
    e1v           (

In [9]:
np.shape(sci_dom.dataset.e3t_0.cumsum(dim='z').squeeze())

(51, 375, 297)

In [12]:
def difftpt2tpt(var,dim):
    """
    Compute the Euler derivative of T-pt variable onto a T-pt.
    Input the dimension index for derivative
    """
    return  0.5*( var.roll(dim=-1, roll_coords=True) - var.roll(dim=+1, roll_coords=True) )


In [13]:
tt = difftpt2tpt( sci_nwes.dataset.votemper, deptht )


NameError: name 'deptht' is not defined

In [9]:
def difftpt2tpt(var):
    """
    Compute the Euler derivative of T-pt variable onto a T-pt.
    Input the dimension index for derivative
    """
    return  0.5*( var.roll(deptht=-1, roll_coords=True) - var.roll(deptht=+1, roll_coords=True) )


In [ ]:
tt = difftpt2tpt( sci_nwes.dataset.votemper)


In [8]:
t

<xarray.DataArray 'votemper' (time_counter: 7, deptht: 51, y: 226, x: 136)>
[10972752 values with dtype=float32]
Coordinates:
  * deptht        (deptht) float32 3.038086 9.3671875 ... 5617.8516 5822.174
    nav_lat       (y, x) float32 ...
    nav_lon       (y, x) float32 ...
  * time_counter  (time_counter) datetime64[ns] 2007-01-01T11:58:56 ... 2007-01-31T11:58:56
    time_instant  (time_counter) datetime64[ns] ...
Dimensions without coordinates: y, x
Attributes:
    units:               degC
    online_operation:    instant
    interval_operation:  300 s
    interval_write:      1 d
    cell_methods:        time: point (interval: 300 s)